# Deep Q-Networks (DQNs)

In [3]:
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

In [4]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1

In [5]:
!pip install gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow


In [8]:
import gym
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque


In [9]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [10]:
env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [12]:
episodes = 100
batch_size = 32
total_rewards = []

for episode in range(1, episodes + 1):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    time = 0
    episode_reward = 0
    
    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        time += 1
        episode_reward += reward

        if done:
            print(f"Episode: {episode}/{episodes}, Score: {time}, Epsilon: {agent.epsilon:.2}")
            break
    total_rewards.append(episode_reward)
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 20ms/step
Episode: 35/100, Score: 16, Epsilon: 0.75
1/1 [==============================] - 0s 19ms/step
Episode: 36/100, Score: 38, Epsilon: 0.75
1/1 [==============================] - 0s 25ms/step
Episode: 37/100, Score: 11, Epsilon: 0.74
1/1 [==============================] - 0s 21ms/step
Episode: 38/100, Score: 11, Epsilon: 0.74
1/1 [==============================] - 0s 29ms/step
Episode: 39/100, Score: 16, Epsilon: 0.74
1/1 [==============================] - 0s 19ms/step
Episode: 40/100, Score: 14, Epsilon: 0.73
1/1 [==============================] - 0s 32ms/step
Episode: 41/100, Score: 13, Epsilon: 0.73
1/1 [==============================] - 0s 19ms/step
Episode: 42/100, Score: 21, Epsilon: 0.73
1/1 [==============================] - 0s 33ms/step
Episode: 43/100, Score: 16, Epsilon: 0.72
1/1 [==============================] - 0s 25ms/step
Episode: 44/100, Score: 13, Epsilon: 0.72
1/1 [====

In [13]:
average_score = np.mean(total_rewards)
print(f"Average Score over {episodes} Evaluation Episodes: {average_score}")

Average Score over 100 Evaluation Episodes: 15.68
